# Dataset

In [ ]:
# from geometric_svmest import *
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


def is_collision(x):
    R_OBS = 0.8
    return np.linalg.norm(x, axis=1) <= R_OBS


# N = 10000000, 6dof trained in 3mn50s on laptop
N_TRAIN = 10000000
dof = 6
X_train = np.random.uniform(-np.pi, np.pi, size=(N_TRAIN, dof))
# y_train = np.where(is_collision(X_train), -1, +1)  # -1: collision, +1: free
y_train = np.where(is_collision(X_train), +1, -1)  # -1: free, +1: collision
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")

In [ ]:
sigma = 0.5
model = SVC(kernel="rbf", gamma=1 / (2 * sigma**2), C=1.0)

model.fit(X_train, y_train)
fhater = model.decision_function

In [ ]:
X_test = np.random.uniform(-np.pi, np.pi, size=(1000000, dof))
# y_test = np.where(is_collision(X_test), -1, +1)
y_test = np.where(is_collision(X_test), +1, -1)
y_pred = model.predict(X_test)
accscore = accuracy_score(y_test, y_pred)
print(f"Test accuracy: {accscore*100:.2f}%")

In [ ]:
supvecs = model.support_vectors_
supvecs_labels = y_train[model.support_]
supvecs_free = supvecs[supvecs_labels == +1]
supvecs_cols = supvecs[supvecs_labels == -1]
print(f"Number of support vectors: {len(supvecs)}")
print(f"Number of free support vectors: {len(supvecs_free)}")
print(f"Number of collision support vectors: {len(supvecs_cols)}")

print(supvecs)

In [ ]:
x0 = np.array([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]])
pred = model.predict(x0)
predf = fhater(x0)
print(f"Prediction for {x0}: {pred}")
print(f"Decision function value for {x0}: {predf}")

x1 = np.array([[3.0, 3.0, 3.0, 3.0, 3.0, 3.0]])
pred1 = model.predict(x1)
predf1 = fhater(x1)
print(f"Prediction for {x1}: {pred1}")
print(f"Decision function value for {x1}: {predf1}")

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances


def adjacency_euclidean_mid_penalty(X):
    Weudist = euclidean_distances(X, X)
    print(Weudist.shape)

    Wcenter = []
    Wcenterid = []
    for i in range(len(Weudist)):
        for j in range(i + 1, len(Weudist)):
            Wcenter.append(0.5 * (X[i] + X[j]))
            Wcenterid.append((i, j))
    Wcenter = np.array(Wcenter)
    Wcenterid = np.array(Wcenterid)
    print(Wcenter.shape)
    print(Wcenterid.shape)


    Wcenterf = fhater(Wcenter)
    Wcenterclass = model.predict(Wcenter)
    print(Wcenterf.shape)
    print(Wcenterclass.shape)
    print(Wcenterclass)


adjacency_euclidean_mid_penalty(supvecs)

In [ ]:
from sklearn.metrics.pairwise import rbf_kernel


def soft_adjacency_mid_penalty(X, f):
    W = rbf_kernel(X, X, gamma=1 / (2 * sigma**2))
    conf = 1 / (1 + np.exp(-f))  # sigmoid
    W *= np.minimum(conf[:, None], conf[None, :])

    # centers = (X_nodes[:, None, :] + X_nodes[None, :, :]) / 2
    # midpoints_flat = centers.reshape(-1, 2)
    # values = f_hat(midpoints_flat).reshape(len(X_nodes), len(X_nodes))
    # p = 1 / (1 + np.exp(-values))
    # W *= p
    # must optimize later for memory efficiency
    for i in range(len(X)):
        for j in range(i + 1, len(X)):
            xm = 0.5 * (X[i] + X[j])
            xmfhat = fhater(xm.reshape(1, -1))[0]
            p = 1 / (1 + np.exp(-xmfhat))
            W[i, j] *= p
            W[j, i] *= p

    np.fill_diagonal(W, 0)
    return W


X_nodes = supvecs_free
f_nodes = fhater(X_nodes)
print(f"X_nodes: {X_nodes.shape}")
print(f"f_nodes: {f_nodes.shape}")
W = soft_adjacency_mid_penalty(X_nodes, f_nodes)

In [ ]:
start = np.array([-2.5, -2.5, -2.5, -2.5, -2.5, -2.5])
goal = np.array([2.5, 2.5, 2.5, 2.5, 2.5, 2.5])

fstart = fhater(start.reshape(1, -1))[0]
fgoal = fhater(goal.reshape(1, -1))[0]

X_all = np.vstack([start, goal, X_nodes])
f_all = np.hstack([fstart, fgoal, f_nodes])

W_all = soft_adjacency_mid_penalty(X_all, f_all)
C = 1 / (W_all + 1e-6)  # cost matrix

In [ ]:
from scipy.sparse.csgraph import shortest_path

dist, pred = shortest_path(C, return_predecessors=True)
path = []
j = 1  # goal index
while j != -9999:
    path.append(j)
    j = pred[0, j]
path = path[::-1]

pathlength = 0.0
for i in range(len(path) - 1):
    pathlength += np.linalg.norm(X_all[path[i + 1]] - X_all[path[i]])

pathq = X_all[path]

print(f"Path indices: {path}")
print(f"Path length: {pathlength:.2f}")
print(f"Path configurations:\n{pathq}")
